# DSPY
DSPy is the framework for programming language models. It allows you to iterate fast on building modular AI systems and offers algorithms for optimizing their prompts and weights, whether you're building simple classifiers, sophisticated RAG pipelines, or Agent loops.

You write compositional Python code and use DSPy to teach your LM to deliver high-quality outputs.

# Setup

In [10]:
from configure import configure_dspy_local_with_ollama

ollama_model = "llama3.2:1b"
configure_dspy_local_with_ollama(ollama_model)

Successfully pulled model: llama3.2:1b
Output: 


In [19]:
import dspy

math = dspy.Predict("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")

APIError: litellm.APIError: AzureException APIError - argument of type 'NoneType' is not iterable

In [8]:
import dspy
import dsp

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.configure(rm=colbertv2_wiki17_abstracts)
retrieval_response = dsp.retrieve("When was the second FIFA World Cup held?", k=5)

for result in retrieval_response:
    print("Text:", result, "\n")

Text: 1934 FIFA World Cup | The 1934 FIFA World Cup was the second FIFA World Cup, the world championship for men's national association football teams. It took place in Italy from 27 May to 10 June 1934. 

Text: 1990 FIFA World Cup | The 1990 FIFA World Cup was the 14th FIFA World Cup, the quadrennial international football world championship tournament. It was held from 8 June to 8 July 1990 in Italy, the second country to host the event twice (the first being Mexico in 1986). Teams representing 116 national football associations entered, and qualification began in April 1988. A total of 22 teams qualified from this process, along with host nation Italy and defending champion Argentina. 

Text: 1998 FIFA World Cup | The 1998 FIFA World Cup was the 16th FIFA World Cup, the world championship for men's national association football teams. It was held in France from 10 June to 12 July 1998. The country was chosen as the host nation by FIFA for the second time in the history of the tourn

In [54]:
import dspy

lm = dspy.LM(
    model="azure/gpt-4o",
    api_key="8854aab4a6084397ae1501c9da395170",
    api_base="https://en-oai-sweden-c.openai.azure.com/",
    api_version="2024-08-01-preview",
    cache=False,
    # cache=False
)
dspy.configure(lm=lm)


In [55]:
react = dspy.Predict("question -> answer")
react(question="Who is David Beckham?")

Prediction(
    answer='David Beckham is a retired English professional footballer, businessman, and current president and co-owner of Inter Miami CF. He is widely regarded as one of the greatest midfielders of his generation and is known for his exceptional passing, crossing ability, and bending free-kicks. Beckham played for several top clubs, including Manchester United, Real Madrid, LA Galaxy, AC Milan, and Paris Saint-Germain. He also earned 115 caps for the England national team, serving as captain for six years. Off the field, Beckham is a global ambassador for the sport and has a significant influence in fashion and popular culture.'
)

In [46]:
react(question="How old is he?")

Prediction(
    answer='I\'m sorry, I need more context to determine who "he" is and provide an age. Could you please provide more details?'
)

In [2]:
import mlflow

mlflow.dspy.autolog()

# This is optional. Create an MLflow Experiment to store and organize your traces.
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='file:///Users/erannahum/Projects/dspy-tutorial/mlruns/608625338185211140', creation_time=1733292059750, experiment_id='608625338185211140', last_update_time=1733292059750, lifecycle_stage='active', name='DSPy', tags={}>

In [14]:
import dspy
        
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer", tools=[evaluate_math, search_wikipedia])


In [18]:

pred = react(question="Which baseball team does Shohei Ohtani play for?")


In [16]:
pred

Prediction(
    trajectory={'thought_0': 'To find out which baseball team Shohei Ohtani plays for, I will use the search_wikipedia tool to look up information on Shohei Ohtani.', 'tool_name_0': 'search_wikipedia', 'tool_args_0': {'query': 'Shohei Ohtani'}, 'observation_0': ["Shohei Otani | Shohei Otani (大谷 翔平 , Ohtani Shōhei , born July 5, 1994) is a Japanese baseball pitcher and outfielder who currently plays for the Hokkaido Nippon-Ham Fighters of Nippon Professional Baseball's (NPB) Pacific League. He was the first pick of the Fighters in the 2012 draft. He has officially recorded the fastest pitch by a Japanese pitcher and in NPB history at 165 km/h .", 'Shōhei Ōoka | Shōhei Ōoka (大岡 昇平 , Ōoka Shōhei , 6 March 1909 – 25 December 1988) was a Japanese novelist, literary critic, and translator of French literature who was active during the Shōwa period of Japan. Ōoka belongs to the group of postwar writers whose World War II experiences at home and abroad figure prominently in their w

In [6]:
dspy.inspect_history(n=5)






[2024-12-04T08:11:59.628674]

System message:

Your input fields are:
1. `question` (str)
2. `trajectory` (str)

Your output fields are:
1. `next_thought` (str)
2. `next_tool_name` (Literal[evaluate_math, search_wikipedia, finish])
3. `next_tool_args` (dict[str, Any])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must be one of: evaluate_math; search_wikipedia; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "object"}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.
        
        You will be given `question` and your goal is to finish with `